In [1]:
#librerias a utilizar
#libreria parse XML
import xml.etree.ElementTree as ET
#expresiones regulares
import re
#pandas para dataframes
import pandas as pd
#acceder a path de directorios locales
import os

In [3]:
#listar los objetos disponibles en el directorio
list_fact=os.listdir()
list_fact

['.ipynb_checkpoints',
 'Ejemplo2Fac.xml',
 'EjemploFac.xml',
 'extraer_datos_facturaXML_SRI.ipynb']

In [7]:
#quedarse con uno solo de los archivos
factura=list_fact[1]
#generar el root de la fuente de datos
root=ET.parse(factura).getroot()

In [10]:
#contenido de root (arhivo raiz de XML)
root
type(root)

xml.etree.ElementTree.Element

In [13]:
#verificar las etiquetas child de root y el atrbiuto
#al final nos quedamos con el texto de comprobante

for i in root:
    print(i.tag,":", i.attrib)

estado : {}
numeroAutorizacion : {}
fechaAutorizacion : {}
ambiente : {}
comprobante : {}
mensajes : {}


In [16]:
#seleccionar solo comprobante
#obtener el texto de comprobante
#abrir el texto de comprobante como XML
comprobante=root[4].text


In [22]:
#usando el metodo de fromstring abres el texto como XML
xml_comprobante=ET.fromstring(comprobante)
xml_comprobante.tag

'factura'

In [26]:
#iterar sobre los tags de factura
#generar un diccionario para ver los datos
data={}
for x in range(len(xml_comprobante)):
    for j in xml_comprobante[x]:
        data[j.tag]=j.text 

In [27]:
data

{'ambiente': '2',
 'tipoEmision': '1',
 'razonSocial': 'COMERCIAL Comercial e Industrial S.A.',
 'nombreComercial': 'COMERCIAL Comercial e Industrial S.A.',
 'ruc': '1790040275001',
 'claveAcceso': '2402202101179004027500120231010001808430085886517',
 'codDoc': '01',
 'estab': '023',
 'ptoEmi': '101',
 'secuencial': '000180843',
 'dirMatriz': 'Av. Colón E4-81 y 9 de Octubre',
 'fechaEmision': '24/03/2021',
 'dirEstablecimiento': 'CALLE 1741 Y AV.CALLE C.C CALLE',
 'contribuyenteEspecial': '5368',
 'obligadoContabilidad': 'SI',
 'tipoIdentificacionComprador': '05',
 'razonSocialComprador': 'PULLUTASIG ACOSTA FERNANDO',
 'identificacionComprador': '9999999999999',
 'totalSinImpuestos': '16.06',
 'totalDescuento': '0.00',
 'totalConImpuestos': None,
 'propina': '0.00',
 'importeTotal': '17.99',
 'moneda': 'DOLAR',
 'pagos': None,
 'detalle': None,
 'campoAdicional': 'AV BOLIVARIANA Y TOLA|999994024|2021-02-24 valor: 17.9900|23-FP-858865||',
 '{http://www.w3.org/2000/09/xmldsig#}SignedInfo

In [39]:
#facturas de las que se va extraer informacion
#son las facturas que listamos
#selecciono las dos facturas ejemplo 
facturas=list_fact[1:3]
facturas

['Ejemplo2Fac.xml', 'EjemploFac.xml']

In [32]:
#funcion para obtener datos de cada comprobante
def obtener_dato(comprobante):
    #generar nuevos xml de parte datos texto
    comprobante=ET.fromstring(comprobante)
    #diccionario a generar
    data={}
        #datos externos de comprobante
    for x in range(len(comprobante)):
        if x<3:
            for j in comprobante[x]:
                data[j.tag]=j.text        
    return data

In [47]:
#funcion para obtener las facturas del listado
def parser(factura):
    root=ET.parse(factura).getroot()
    for i in root:
        if i.tag=='comprobante':
            comprobante=i.text
        
    return comprobante

In [50]:
#usando las funciones para obtener datos
#facturas seleccionadas 
data_df=[]
for i in range(len(facturas)):
    textos=parser(facturas[i])
    data_df.append(obtener_dato(textos))

In [49]:
#generar dataframe usando pandas
df=pd.DataFrame(data_df)
df

,ambiente,tipoEmision,razonSocial,nombreComercial,ruc,claveAcceso,codDoc,estab,ptoEmi,secuencial,...,razonSocialComprador,identificacionComprador,totalSinImpuestos,totalDescuento,totalConImpuestos,propina,importeTotal,moneda,pagos,detalle
0,2,1,COMERCIAL Comercial e Industrial S.A.,COMERCIAL Comercial e Industrial S.A.,1790040275001,2402202101179004027500120231010001808430085886517,01,023,101,000180843,...,PULLUTASIG ACOSTA FERNANDO,9999999999999,16.06,0.00,None,0.00,17.99,DOLAR,None,None
1,2,1,PA-CO Comercial e Industrial S.A.,PA-CO Comercial e Industrial S.A.,1790040275001,2402202101179004027500120231010001808430085886517,01,023,101,000180843,...,PULLUTASIG ACOSTA FERNANDO,9999999999999,16.06,0.00,None,0.00,17.99,DOLAR,None,None
